# Experiments

In [2]:
import numpy as np
import sys
import scipy
import math
import json
import time
import datetime
import matplotlib.pyplot as plt

In [3]:
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, Aer, execute, IBMQ
from qiskit.circuit import Parameter, Gate
from qiskit.tools.jupyter import *
from qiskit.visualization import *
%config InlineBackend.figure_format = 'svg'

from qiskit import QuantumRegister, ClassicalRegister
from qiskit.providers.ibmq import least_busy
from qiskit import visualization

In [4]:
from env import *
import utils
import qutils

In [5]:
def custom_u_h(numberOfQubits):
    qc = QuantumCircuit(numberOfQubits, name='custom_u_h')
    for i in range(numberOfQubits):
        qc.h(i)
    
    return qc

def custom_hcrzch(numberOfQubits, parameter):
    qc = QuantumCircuit(numberOfQubits, name='custom_hcrzch')
    
    for i in range(numberOfQubits):
        qc.h(i)
    
    last_qubit_index = numberOfQubits - 1
    for i in range(last_qubit_index):
        qc.cx(i, i + 1)
    
    qc.rz(parameter, last_qubit_index)
    
    for i in reversed(range(last_qubit_index)):
        qc.cx(i, i + 1)
    
    for i in range(numberOfQubits):
        qc.h(i)
    
    return qc

def custom_success_reflection(numberOfQubits):
    qc = QuantumCircuit(numberOfQubits, name='custom_success_reflection')
    qc.z(0)
    return qc

def custom_reflection_start_state(numberOfQubits):
    qc = QuantumCircuit(numberOfQubits, name='custom_reflection_start_state')
    last_qubit_index = numberOfQubits - 1
    for j in range(numberOfQubits):
            qc.x(j)
        
    qc.h(last_qubit_index)
    qc.mct([qc.qubits[qi] for qi in range(last_qubit_index)],
            qc.qubits[last_qubit_index],
            None,
            mode="noancilla")
    qc.h(last_qubit_index)
        
    for j in range(numberOfQubits):
        qc.x(j)
        
    return qc

def amplitude_circuit(numberOfQubits, unitary, success_reflection_unitary, iterations, barriers=True):
    qubits_range = range(numberOfQubits)
    circuit = QuantumCircuit(numberOfQubits, numberOfQubits)
    circuit.append(unitary.to_instruction(), qubits_range)
    
    reflection_start_state = custom_reflection_start_state(numberOfQubits)
    for i in range(iterations):
        if barriers: circuit.barrier()
        circuit.append(success_reflection_unitary.to_instruction(), qubits_range)
        if barriers: circuit.barrier()
        circuit.append(unitary.inverse().to_instruction(), qubits_range)
        if barriers: circuit.barrier()
        circuit.append(reflection_start_state.to_instruction(), qubits_range)
        if barriers: circuit.barrier()
        circuit.append(unitary.to_instruction(), qubits_range)
    
    return circuit

## Run

In [6]:
def runZeroExperiment(device, shots):
    circuit = circuit_zero()
    transpiled_qc = qutils.optimize(circuit, device)
    qobj = qutils.build(transpiled_qc, device, shots)
    parameters = None
    path = '' if SKIP_RUN else qutils.run(qobj, device, parameters, "zero_experiment")
    return circuit, transpiled_qc, path

def runThetaExperiment(circuit, device, theta, values, shots, comment = ""):
    transpiled_qc = qutils.optimize(circuit, device)
    circuits = [transpiled_qc.bind_parameters({theta: n}) for n in values]
    
    qobj = qutils.build(circuits, device, shots)
    parameters = {str(theta): values}
    path = '' if SKIP_RUN else qutils.run(qobj, device, parameters, comment)
    return circuit, transpiled_qc, path

def runExperiment(circuit, device, shots, comment = ""):
    transpiled_qc = qutils.optimize(circuit, device)
    qobj = qutils.build(transpiled_qc, device, shots)
    parameters = {}
    path = '' if SKIP_RUN else qutils.run(qobj, device, parameters, comment)
    return circuit, transpiled_qc, path

# Experiments

## Amplified

### parametrized

In [7]:
SKIP_RUN = True
nq = 2
a = custom_hcrzch(nq, THETA)
b = custom_success_reflection(nq)

iterations = 0
qc = amplitude_circuit(nq, a, b, iterations, False)
#qc.barrier()
qc.measure([i for i in range(nq)], [i for i in range(nq)])
COMMENT = "Parametrized Amplified Experiment on " + str(nq) + " qubits, noancilla, measurements on all qubits, " + str(iterations) + "iterations"
a_circuit, a_transpiled, a_path = runThetaExperiment(qc,
                                                     CURRENT_DEVICE,
                                                     THETA,
                                                     THETA_VALUES,
                                                     SHOTS,
                                                     COMMENT)

a_files_path = a_path.replace(".json", "")
a_circuit.decompose().draw(output="mpl", filename=a_files_path + ".pdf")
a_transpiled.decompose().draw(output="mpl", filename=a_files_path + "_transpiled.pdf")
print(a_files_path)